# Reading and mutating spark dataframes

In [1]:
from pyspark.sql import SparkSession  # we import from SQL so we can make direct SQL queries
from pathlib import Path

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

21/07/08 16:16:44 WARN Utils: Your hostname, GBLON1WLZ13699 resolves to a loopback address: 127.0.1.1; using 10.164.5.249 instead (on interface eth2)
21/07/08 16:16:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/08 16:16:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data_dir = Path('../pyspark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames')

In [4]:
data_path = str(data_dir/'people.json')

In [5]:
df = spark.read.json(data_path)

In [6]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



Spark can do schema inference

In [8]:
df.columns

['age', 'name']

In [9]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



## Schema specification

Sometimes, for complicated data, the schema inference won't work and we need to manually specify it.

In [10]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [11]:
data_schema = [StructField('age', IntegerType(), True),  # the third argument is if the quantity can be null
               StructField('name', StringType(), True),
              ]  

In [12]:
final_struct = StructType(fields=data_schema)

In [13]:
df = spark.read.json(data_path, schema=final_struct)

In [14]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



---

## Selecting data

Just getting columns gives a column object

In [20]:
type(df['age'])

pyspark.sql.column.Column

But using `select` to get a dataframe object is often more flexible 

In [21]:
df.select('age')

DataFrame[age: int]

In [22]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [32]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



## Mutating dataframes

Adding a column

In [34]:
df.withColumn('double_age', df['age']*2).show()  # needs a column object here, not a dataframe

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



But this is not an inplace operation, would need to save to a new dataframe to retain it.

In [35]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



Renaming a column

In [36]:
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



## Using SQL to interact with a dataframe

Register the dataframe as a SQL temporary view

In [41]:
df.createOrReplaceTempView('people')  # table name

In [43]:
results = spark.sql("SELECT * FROM people WHERE age > 20")

In [44]:
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



So we can use SQL syntax with spark